# 건영이의 알고리즘

In [1]:
# 모듈 로딩
from station_gy import Station_gy
from userInfo_gy import UserInfo
from geopy.geocoders import Nominatim
from settings import *
import numpy as np
import pymysql
import pandas as pd

## [1] geopy모듈

In [2]:
# geopy모듈 확인해보기(geocode)
geolocoder = Nominatim(user_agent = 'South Korea', timeout= 200)
geo = geolocoder.geocode('대구광역시 북구 학정동 학남로 60')
print(geo[0]) # 주소 데이터
print(geo[1]) # 위도 경도
print(geo.latitude) # 위도
print(geo.longitude) # 경도
print(type(geo))

학남로, 국우동, 북구, 대구, 41420, 대한민국
(35.9491887, 128.5685799)
35.9491887
128.5685799
<class 'geopy.location.Location'>


In [3]:
# geopy모듈 확인해보기(reverse)
geolocoder = Nominatim(user_agent = 'South Korea', timeout= 200)
geo = geolocoder.reverse('35.9475,128.5694')
print(geo[0]) # 주소 데이터
print(geo[1]) # 위도 경도
print(geo.latitude) # 위도
print(geo.longitude) # 경도
print(type(geo))

학정초등학교, 학남로, 국우동, 북구, 대구, 41420, 대한민국
(35.94834025, 128.56911985305533)
35.94834025
128.56911985305533
<class 'geopy.location.Location'>


In [4]:
# userinfo모듈을 이용한 유저 위치 입력 => 위도 경도 위치 반환
user1 = UserInfo.geocoding('대구광역시 북구 학정동 학남로 60')
print(user1.loc[0])
print(user1.loc[1])
user1.get_user_info()

35.9491887
128.5685799
사용자 현재 위치 : 대구광역시 북구 학정동 학남로 60
위도 : 35.9491887, 경도 : 128.5685799


In [5]:
# userinfo모듈을 이용한 유저 위치 입력 => 위도 경도 위치 반환
user1 = UserInfo.geocoding_reverse('35.9475, 128.5694')
print(user1.loc[0])
print(user1.loc[1])
user1.get_user_info()

35.9475
128.5694
사용자 현재 위치 : 학정초등학교, 학남로, 국우동, 북구, 대구, 41420, 대한민국
위도 : 35.9475, 경도 : 128.5694


## [2] 파일 불러오기

In [6]:
# Staion이라는 객체 생성
station_app = Station_gy()

In [7]:
# sql에 있는 전체 테이블 이름을 적어야함
station_app.get_table_names()

# 한글명으로 되어 있는 것들은 지금 우리 한테는 별 필요 없는 것이라서 신경쓰지말고 진행하자

['car_register', 'charge_address', 'count', 'seoul_loc', 'speed', '구분_명_주소']

In [8]:
# 서울시 전기차 등록현황
car_register_df=station_app.load_DB('car_register')
# car_register_df.to_csv('C:/Users/USER/Desktop/project/12.계단조/Data/car_register.csv',index=False,encoding='cp949')

In [9]:
# 년도별 서울시 전기차 수
count_df=station_app.load_DB('count')
# count_df.to_csv('C:/Users/USER/Desktop/project/12.계단조/Data/count.csv',index=False,encoding='cp949')

In [10]:
# 서울시 충전소 주소
charge_address_df=station_app.load_DB('charge_address')
# charge_address_df.to_csv('C:/Users/USER/Desktop/project/12.계단조/Data/charge_address.csv',index=False,encoding='cp949')

In [11]:
# 서울시 충전소 위도경도
seoul_loc_df=station_app.load_DB('seoul_loc')
# seoul_loc_df.to_csv('C:/Users/USER/Desktop/project/12.계단조/Data/seoul_loc.csv',index=False,encoding='cp949')

In [12]:
# 서울시 충전소 구분
speed_df=station_app.load_DB('speed')
# speed_df.to_csv('C:/Users/USER/Desktop/project/12.계단조/Data/speed.csv',index=False,encoding='cp949')

## [3] 유클리드 거리 구하기

In [13]:
from haversine import haversine

In [14]:
sta=UserInfo.geocoding('대구광역시 북구 대학로 80').loc # 출발 장소
end=UserInfo.geocoding('대구광역시 동구 공항로 221').loc # 도착 장소

haversine(sta,end)

# 출방장소와 도착장소는 2.2824200352130695km만큼 떨어져 있다.

2.2824200352130695

In [15]:
# 5km
a=np.array((35.882,128.6623))
b=np.array((35.8708,128.7157))

haversine(a,b)

4.969880820326101

In [16]:
# 3km
a=(35.882, 128.6623)
b=(35.874, 128.6939)

haversine(a,b)

2.9828207457072033

In [17]:
# 모듈을 이용하여 거리 구하기
result=station_app.cal_dis(sta,end)
result

2.2824200352130695

## [4] 알고리즘 구현

### [4-1] 충전소 데프 만들기
- 사용자 3/5/7km내의 충전소 정보 제공

In [18]:
# 충전소 객체를 만들고 새로운 데이터 프레임 만들기
station_gy=Station_gy(use_DB=True)
sta=UserInfo.geocoding('서울특별시 영등포구 여의동로 330').loc # 출발 장소: 여의도 공원
result=station_gy.station_df(sta)
result

,station,LAT,LNG,dis(km),address,speed
0,남서울직할,37.527292,126.929853,0.551920,서울특별시 영등포구 여의도동 21,완속/급속
1,롯데캐슬엠파이어,37.520241,126.926409,0.932338,서울특별시 영등포구 여의도동 의사당대로 127,완속/급속


### [4-2] 등록차량 데프 만들기
- 동일 구내의 전기차의 수를 반환

In [19]:
# 서울시 전기차 등록현황
car_register_df=station_app.load_DB('car_register')
car_register_df

,ind,dong,fuel
0,1,서울특별시 강남구 개포1동,하이브리드(휘발유+전기)
1,2,서울특별시 강남구 개포1동,하이브리드(휘발유+전기)
2,3,서울특별시 강남구 개포2동,하이브리드(휘발유+전기)
3,4,서울특별시 강남구 개포2동,하이브리드(휘발유+전기)
4,5,서울특별시 강남구 개포1동,하이브리드(휘발유+전기)
...,...,...,...
189570,189571,서울특별시 강남구 개포2동,전기
189571,189572,서울특별시 강남구 개포2동,하이브리드(휘발유+전기)
189572,189573,서울특별시 강남구 개포2동,하이브리드(휘발유+전기)
189573,189574,서울특별시 강남구 개포2동,하이브리드(휘발유+전기)


In [20]:
sta1=UserInfo.geocoding('서울특별시 영등포구 여의동로 330')
gu=sta1.address.split(' ')[1]
print(gu)

영등포구


In [21]:
c=station_gy.res_car_cnt(sta1.address)
print(c)

           ind              dong           fuel
23          24   서울특별시 영등포구 당산1동  하이브리드(휘발유+전기)
29          30   서울특별시 영등포구 당산1동  하이브리드(휘발유+전기)
80          81   서울특별시 영등포구 당산1동             전기
85          86   서울특별시 영등포구 당산1동             수소
88          89   서울특별시 영등포구 당산1동  하이브리드(휘발유+전기)
...        ...               ...            ...
188733  188734  서울특별시 영등포구 영등포본동             전기
188734  188735  서울특별시 영등포구 영등포본동  하이브리드(휘발유+전기)
188735  188736  서울특별시 영등포구 영등포본동             전기
188736  188737  서울특별시 영등포구 영등포본동  하이브리드(휘발유+전기)
188737  188738  서울특별시 영등포구 영등포본동             전기

[9275 rows x 3 columns]


In [22]:
station_gy.gu_res_car_cnt(c)

fuel,수소,전기,하이브리드(LPG+전기),하이브리드(경유+전기),하이브리드(휘발유+전기)
count,159,2388,102,88,6538


In [23]:
gu_res_car_df=pd.DataFrame(c.groupby('fuel').count()['dong'])
a=gu_res_car_df.T
a.index=['count']
a

fuel,수소,전기,하이브리드(LPG+전기),하이브리드(경유+전기),하이브리드(휘발유+전기)
count,159,2388,102,88,6538


In [24]:
conn = pymysql.connect(host=host_IP, user=user_ID, password=password, db=db_name, charset=charset) # mysql과 연결
cur = conn.cursor() # Cursor객체를 가기고 옴
sql = f"SELECT * FROM count"
cur.execute(sql) # sql문장을 DB서버에 전송
rows = cur.fetchall() # 서버로부터 가져온 데이터를 코드에서 활용 => fetchall(), fetchone(), fetchmany() 등을 이용함
tabel = pd.DataFrame(rows, columns = [t[0] for t in cur.description])
cur.close()
conn.close()
print(tabel)

    year  counts
0   2005       6
1   2006      24
2   2007     122
3   2008     207
4   2009     779
5   2010     960
6   2011    2348
7   2012    4913
8   2013    4015
9   2014    5306
10  2015    6597
11  2016   10690
12  2017   16469
13  2018   20916
14  2019   23576
15  2020   35929
16  2021   50847
17  2022    5882


In [25]:
cur.description

(('year', 253, None, 40, 40, 0, False), ('counts', 3, None, 11, 11, 0, True))

- description => 각 칼럼의 특징 알아보기 (필드명,데이터형_코드, 표시크기, 내부크기, 정확도, 비율, nullable)

In [26]:
conn=pymysql.connect(host=host_IP,user=user_ID,password=password,db=db_name,charset=charset)
cur=conn.cursor()
sql=f'show tables in {db_name}'
cur.execute(sql)
tb_name=cur.fetchall()
tablelist=[tb[0] for tb in tb_name]
cur.close()
conn.close()
tablelist

['car_register', 'charge_address', 'count', 'seoul_loc', 'speed', '구분_명_주소']

In [27]:
tb_name

(('car_register',),
 ('charge_address',),
 ('count',),
 ('seoul_loc',),
 ('speed',),
 ('구분_명_주소',))

In [28]:
for a in rows:
    print(a[0])

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
